# 🎬 MOVIMENTO REAL: Forçar Mudanças Dramáticas

**Problema identificado:**
- ❌ Stable Diffusion muito conservador
- ❌ Apenas pequenas variações, sem movimento real
- ❌ Strength baixo demais (0.3-0.5)

**Solução:**
- ✅ Strength ALTO (0.7-0.8) para mudanças dramáticas
- ✅ Prompts que FORÇAM movimento específico
- ✅ Sequência de poses completamente diferentes
- ✅ Usar txt2img como fallback para mudanças maiores
- ✅ Interpolação entre poses extremas


In [ ]:
# Instalação
%pip -q install diffusers==0.30.0 transformers accelerate safetensors "imageio[ffmpeg]" pillow


In [ ]:
# 🎬 MOVIMENTO DRAMÁTICO: Strength ALTO + Poses Extremas
import os, numpy as np
from PIL import Image
import torch, imageio
from diffusers import StableDiffusionImg2ImgPipeline, StableDiffusionPipeline, DPMSolverMultistepScheduler
import time

print("🎬 Iniciando MOVIMENTO DRAMÁTICO do Homer...")
start_time = time.time()

# Configuração para MOVIMENTO REAL
model_id = "runwayml/stable-diffusion-v1-5"
width, height = 512, 512
width  = (width  // 8) * 8
height = (height // 8) * 8
steps, guidance, frames, fps = 20, 7.5, 8, 12
strength = 0.75  # ← MUITO ALTO para forçar mudanças dramáticas
negative_prompt = "blurry, low quality, distorted, deformed, ugly, disfigured, mutated, extra limbs, missing limbs, bad anatomy, bad proportions, malformed, different character, not Homer Simpson"

print(f"📊 Config: {width}x{height}, {steps} steps, {frames} frames, strength={strength}")

# Carregar Homer
img_path = "/Users/alansms/stable_diffusion_video_generator/homer.png"
if not os.path.exists(img_path):
    raise FileNotFoundError(f"Imagem não encontrada: {img_path}")
init_img = Image.open(img_path).convert("RGB").resize((width, height), Image.LANCZOS)
print(f"✅ Homer carregado: {init_img.size}")

# Pipeline Img2Img + Txt2Img
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)
pipe_txt2img = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)

# Scheduler
pipe_img2img.scheduler = DPMSolverMultistepScheduler.from_config(pipe_img2img.scheduler.config)
pipe_txt2img.scheduler = DPMSolverMultistepScheduler.from_config(pipe_txt2img.scheduler.config)
pipe_img2img = pipe_img2img.to(device)
pipe_txt2img = pipe_txt2img.to(device)

if device == "cuda":
    pipe_img2img.enable_attention_slicing()
    pipe_txt2img.enable_attention_slicing()

print("✅ Pipelines prontos")

# SEQUÊNCIA DE POSES EXTREMAS (movimento real)
dramatic_poses = [
    "Homer Simpson, yellow skin, cartoon character, arms down, relaxed pose",
    "Homer Simpson, yellow skin, cartoon character, both arms raised high, excited, jumping pose", 
    "Homer Simpson, yellow skin, cartoon character, pointing forward with right arm, determined",
    "Homer Simpson, yellow skin, cartoon character, hands on hips, confident pose",
    "Homer Simpson, yellow skin, cartoon character, arms crossed, thinking pose",
    "Homer Simpson, yellow skin, cartoon character, both hands on head, surprised, mouth open",
    "Homer Simpson, yellow skin, cartoon character, one arm up, one arm down, dynamic pose",
    "Homer Simpson, yellow skin, cartoon character, original pose, arms raised"
]

frames_list = [init_img]
current = init_img

print(f"🎞️ Gerando {frames} frames com MOVIMENTO DRAMÁTICO...")
for i in range(1, frames):
    frame_start = time.time()
    prompt = dramatic_poses[i-1]
    print(f"  Frame {i+1}/{frames}: {prompt[:40]}...", end=" ")
    
    # Tentar Img2Img com strength alto
    try:
        out = pipe_img2img(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=current,
            strength=strength,  # ← MUITO ALTO para mudanças dramáticas
            num_inference_steps=steps,
            guidance_scale=guidance,
        )
        img = out.images[0]
    except Exception as e:
        print(f"Img2Img falhou, usando Txt2Img: {e}")
        # Fallback para Txt2Img para mudanças mais dramáticas
        out = pipe_txt2img(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=steps,
            guidance_scale=guidance,
        )
        img = out.images[0]
    
    if not isinstance(img, Image.Image):
        img = Image.fromarray(np.array(img)).convert("RGB")
    
    frames_list.append(img)
    current = img
    
    frame_time = time.time() - frame_start
    print(f"✅ ({frame_time:.1f}s)")

# Exportar
os.makedirs("output", exist_ok=True)
out_path = "output/homer_movimento_dramatico.mp4"
with imageio.get_writer(out_path, fps=fps, quality=8) as w:
    for im in frames_list:
        w.append_data(np.array(im.convert("RGB")))

total_time = time.time() - start_time
print(f"🎉 CONCLUÍDO em {total_time:.1f} segundos!")
print(f"✅ Vídeo salvo: {out_path}")
print(f"📊 {len(frames_list)} frames, {fps} FPS, {width}x{height}px")
print(f"⚡ Tempo médio por frame: {total_time/frames:.1f}s")
print("🎬 Resultado: Homer com MOVIMENTO REAL e dramático!")


In [1]:
# 🚀 MÉTODO HÍBRIDO: Txt2Img + Img2Img para movimento máximo
import os, numpy as np
from PIL import Image
import torch, imageio
from diffusers import StableDiffusionImg2ImgPipeline, StableDiffusionPipeline, DPMSolverMultistepScheduler
import time

print("🚀 Iniciando MÉTODO HÍBRIDO para movimento máximo...")
start_time = time.time()

# Configuração híbrida
model_id = "runwayml/stable-diffusion-v1-5"
width, height = 512, 512
width  = (width  // 8) * 8
height = (height // 8) * 8
steps, guidance, frames, fps = 20, 7.5, 6, 12
negative_prompt = "blurry, low quality, distorted, deformed, ugly, disfigured, mutated, extra limbs, missing limbs, bad anatomy, bad proportions, malformed, different character, not Homer Simpson"

print(f"📊 Config: {width}x{height}, {steps} steps, {frames} frames")

# Carregar Homer
img_path = "/Users/alansms/stable_diffusion_video_generator/homer.png"
if not os.path.exists(img_path):
    raise FileNotFoundError(f"Imagem não encontrada: {img_path}")
init_img = Image.open(img_path).convert("RGB").resize((width, height), Image.LANCZOS)
print(f"✅ Homer carregado: {init_img.size}")

# Pipelines
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)
pipe_txt2img = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    safety_checker=None,
    requires_safety_checker=False,
)

pipe_img2img.scheduler = DPMSolverMultistepScheduler.from_config(pipe_img2img.scheduler.config)
pipe_txt2img.scheduler = DPMSolverMultistepScheduler.from_config(pipe_txt2img.scheduler.config)
pipe_img2img = pipe_img2img.to(device)
pipe_txt2img = pipe_txt2img.to(device)

if device == "cuda":
    pipe_img2img.enable_attention_slicing()
    pipe_txt2img.enable_attention_slicing()

print("✅ Pipelines híbridos prontos")

# MÉTODO HÍBRIDO: Alternar entre Txt2Img e Img2Img
hybrid_sequence = [
    ("txt2img", "Homer Simpson, yellow skin, cartoon character, arms down, relaxed pose"),
    ("img2img", "Homer Simpson, yellow skin, cartoon character, both arms raised high, excited, jumping pose", 0.8),
    ("txt2img", "Homer Simpson, yellow skin, cartoon character, pointing forward with right arm, determined"),
    ("img2img", "Homer Simpson, yellow skin, cartoon character, hands on hips, confident pose", 0.7),
    ("txt2img", "Homer Simpson, yellow skin, cartoon character, both hands on head, surprised, mouth open"),
    ("img2img", "Homer Simpson, yellow skin, cartoon character, original pose, arms raised", 0.6)
]

frames_list = [init_img]
current = init_img

print(f"🎞️ Gerando {frames} frames com MÉTODO HÍBRIDO...")
for i in range(1, frames):
    frame_start = time.time()
    method, prompt, *strength_args = hybrid_sequence[i-1]
    strength = strength_args[0] if strength_args else 0.7
    
    print(f"  Frame {i+1}/{frames} ({method}): {prompt[:40]}...", end=" ")
    
    if method == "txt2img":
        # Txt2Img para mudanças mais dramáticas
        out = pipe_txt2img(
            prompt=prompt,
            negative_prompt=negative_prompt,
            num_inference_steps=steps,
            guidance_scale=guidance,
        )
    else:
        # Img2Img com strength alto
        out = pipe_img2img(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=current,
            strength=strength,
            num_inference_steps=steps,
            guidance_scale=guidance,
        )
    
    img = out.images[0]
    if not isinstance(img, Image.Image):
        img = Image.fromarray(np.array(img)).convert("RGB")
    
    frames_list.append(img)
    current = img
    
    frame_time = time.time() - frame_start
    print(f"✅ ({frame_time:.1f}s)")

# Exportar
os.makedirs("output", exist_ok=True)
out_path = "output/homer_hibrido_movimento.mp4"
with imageio.get_writer(out_path, fps=fps, quality=8) as w:
    for im in frames_list:
        w.append_data(np.array(im.convert("RGB")))

total_time = time.time() - start_time
print(f"🎉 CONCLUÍDO em {total_time:.1f} segundos!")
print(f"✅ Vídeo salvo: {out_path}")
print(f"📊 {len(frames_list)} frames, {fps} FPS, {width}x{height}px")
print(f"⚡ Tempo médio por frame: {total_time/frames:.1f}s")
print("🚀 Resultado: Homer com MOVIMENTO MÁXIMO usando método híbrido!")


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 Iniciando MÉTODO HÍBRIDO para movimento máximo...
📊 Config: 512x512, 20 steps, 6 frames
✅ Homer carregado: (512, 512)


Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00, 47.27it/s]


✅ Pipelines híbridos prontos
🎞️ Gerando 6 frames com MÉTODO HÍBRIDO...
  Frame 2/6 (txt2img): Homer Simpson, yellow skin, cartoon char... 

100%|██████████| 20/20 [00:57<00:00,  2.86s/it]


...

✅ (60.4s)
  Frame 5/6 (img2img): Homer Simpson, yellow skin, cartoon char... 

100%|██████████| 14/14 [00:38<00:00,  2.72s/it]


✅ (45.0s)
  Frame 6/6 (txt2img): Homer Simpson, yellow skin, cartoon char... 

100%|██████████| 20/20 [00:55<00:00,  2.75s/it]


✅ (61.2s)
🎉 CONCLUÍDO em 282.9 segundos!
✅ Vídeo salvo: output/homer_hibrido_movimento.mp4
📊 6 frames, 12 FPS, 512x512px
⚡ Tempo médio por frame: 47.2s
🚀 Resultado: Homer com MOVIMENTO MÁXIMO usando método híbrido!
